# Kerakli kutubxonalarni yuklash

In [8]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
import tensorflow_datasets as tfds

TensorFlow version: 2.10.1


# Datasetni yuklash

In [10]:
dataset = tfds.load("malaria", split = "train")
batch_size = 32

def preprocess_data(example):
    image = example['image']
    image = tf.image.resize(image, [32, 32])
    image = tf.cast(image, tf.float32) / 255.0
    label = example['label']
    return image, label

dataset = dataset.map(preprocess_data)

dataset = dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

# Arxitektura

In [12]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(2)

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)


model = MyModel()

# loss va accuracy

In [13]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

In [14]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

# Train va test functions

In [15]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [16]:
@tf.function
def test_step(images, labels):
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

# Train jarayoni

In [17]:
EPOCHS = 5

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in dataset:
    train_step(images, labels)

  # for test_images, test_labels in test_ds:
  #   test_step(test_images, test_labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
  )

Epoch 1, Loss: 0.6070646047592163, Accuracy: 69.3083724975586, Test Loss: 0.0, Test Accuracy: 0.0
Epoch 2, Loss: 0.42293113470077515, Accuracy: 81.19239044189453, Test Loss: 0.0, Test Accuracy: 0.0
Epoch 3, Loss: 0.307054340839386, Accuracy: 87.46280670166016, Test Loss: 0.0, Test Accuracy: 0.0
Epoch 4, Loss: 0.2408072054386139, Accuracy: 90.63793182373047, Test Loss: 0.0, Test Accuracy: 0.0
Epoch 5, Loss: 0.1869901418685913, Accuracy: 93.08731079101562, Test Loss: 0.0, Test Accuracy: 0.0
